# Plotting: spatial maps

In this notebook, we illustrate the possibilities of plotting 2D spatial maps.

Note that Osyris's plotting functions are wrapping Matplotlib's plotting functions,
and forwards most Matplotlib arguments to the underlying function.

In [ ]:
import osyris
import numpy as np
import matplotlib.pyplot as plt

au = osyris.units("au")

path = "osyrisdata/starformation"
data = osyris.Dataset(8, path=path).load()
ind = np.argmax(data["hydro"]["density"])
center = data["amr"]["position"][ind]

## 2D slice of a scalar quantity

### A 2D slice of gas density

To create a 2D map of a gas density slice 2000 au wide through the plane normal to `z`, use

In [ ]:
osyris.map(
    data["hydro"]["density"], norm="log", dx=2000 * au, origin=center, direction="z"
)

### Rendering modes

By default, the map is rendered as an image, but other rendering modes are available.
The possible modes for scalar quantities are `image`, `contourf` and `contour`.
For example, the same map using `contourf` yields

In [ ]:
osyris.map(
    data["hydro"]["density"],
    norm="log",
    dx=2000 * au,
    origin=center,
    mode="contourf",
    direction="z",
)

### Changing the colorscale

The colormap and the range of values can be changed as follows.

In [ ]:
osyris.map(
    data["hydro"]["density"],
    norm="log",
    dx=2000 * au,
    origin=center,
    direction="z",
    cmap="magma",
    vmin=1.0e-16,
    vmax=1.0e-13,
)

### Controlling the resolution

By default, the maps have a resolution of 256x256 pixels.
This can be changed via the keyword argument `resolution`,
which can either be an integer (in which case the same resolution is applied to both the x and y dimension)
or a dict with the syntax `resolution={'x': nx, 'y': ny}`.

In [ ]:
osyris.map(
    data["hydro"]["density"],
    norm="log",
    dx=2000 * au,
    origin=center,
    direction="z",
    resolution=64,
)

## Maps of vector quantities

By default, if a vector quantity (e.g. gas velocity or magnetic field) is passed to the `map` function,
it will make an image of the magnitude of the vectors.

However, one can also use the `vec`, `stream` and `lic` rendering modes to represent vector Arrays.

### Vector field as arrows (quiver plot)

The `vec` mode for the gas velocity produces

In [ ]:
osyris.map(
    data["hydro"]["velocity"],
    mode="vec",
    dx=2000 * au,
    origin=center,
    direction="z",
    color="k",
)

A colormap can be used to color the arrows:

In [ ]:
osyris.map(
    data["hydro"]["velocity"],
    mode="vec",
    color=data["hydro"]["velocity"],
    cmap="plasma",
    dx=2000.0 * au,
    origin=center,
    direction="z",
)

### Streamlines

Streamlines can be plotted using the `stream` mode:

In [ ]:
osyris.map(
    data["hydro"]["B_field"],
    mode="stream",
    dx=2000 * au,
    origin=center,
    direction="x",
    color="g",
)

A colormap can be used to color the streamlines:

In [ ]:
osyris.map(
    data["hydro"]["velocity"],
    mode="stream",
    color=data["hydro"]["velocity"],
    cmap="jet",
    dx=2000 * au,
    origin=center,
    direction="z",
)

### Line integral convolution

The line integral convolution (LIC) visualizations method, although computationally intensive,
offers an excellent representation of the vector field (especially near discontinuities).

**Note:** This feature requires the [lic](https://pypi.org/project/lic/) package to be installed.

Below is an example of a LIC of the velocity vector field:

In [ ]:
osyris.map(
    {"data": data["hydro"]["velocity"], "mode": "lic"},
    dx=2000 * au,
    direction="z",
    origin=center,
    cmap="binary",
)

Using an `Array` for the colors also works on LICs:

In [ ]:
osyris.map(
    data["hydro"]["velocity"],
    mode="lic",
    color=data["hydro"]["density"],
    norm="log",
    dx=2000 * au,
    origin=center,
    direction="z",
)

## Layers

More than one quantity can be overlayed on the map, using the concept of layers.
Any number of layers can be added.

A layer can be a raw `Array`,
in which case it gets styled according to the arguments passed to the `map` function.
In addition, a layer can also be a `dict`, that needs to contain a `data` entry that holds the `Array`,
and then any other styling members that will apply to that layer and override the global arguments.

For example, plotting a map of gas density with velocity vectors overlayed as arrows can be achieved by doing

In [ ]:
osyris.map(
    {"data": data["hydro"]["density"], "norm": "log"},  # layer 1
    {"data": data["hydro"]["velocity"], "mode": "vec"},  # layer 2
    dx=2000 * au,
    origin=center,
    direction="z",
)

Another example is to overlay contours of the AMR levels onto a map of gas temperature:

In [ ]:
osyris.map(
    {
        "data": data["hydro"]["temperature"],
        "norm": "log",
        "mode": "contourf",
        "levels": np.logspace(0.9, 2, 11),
        "cmap": "hot",
    },  # layer 1
    {
        "data": data["amr"]["level"],
        "mode": "contour",
        "colors": "w",
        "levels": [6, 7, 8],
        "fmt": "%i",
    },  # layer 2
    dx=2000 * au,
    origin=center,
    direction="z",
)

## Arbitrary orientation

In the previous examples, all the maps were created normal to the `z` direction.
The orientation can be defined in multiple ways:

- `'x'`, `'y'`, or `'z'`, to select one of the three cartesian axes as the normal to the plane
- a list of 3 values defining the components of the vector normal to the plane
- `'top'` or `'side'` to choose an automatic top or side view of a disk

### Principal XYZ axes

Viewing the system along the `x` axis is achieved using `direction='x'`:

In [ ]:
osyris.map(
    {"data": data["hydro"]["density"], "norm": "log"},
    {"data": data["hydro"]["velocity"], "mode": "vec"},
    dx=1000 * au,
    origin=center,
    direction="x",
)

### Supplying a normal vector

In this example, we supply a custom vector as the map's `direction`:

In [ ]:
osyris.map(
    {"data": data["hydro"]["density"], "norm": "log"},
    {"data": data["hydro"]["velocity"], "mode": "vec"},
    dx=500 * au,
    origin=center,
    direction=osyris.Vector(-1, 1, 3),
    cmap="magma",
)

### Automatic “top/side” slice orientation according to angular momentum

Osyris also offers an automatic orientation, based on the angular momentum computed inside a region close to the origin.
This is useful for looking at, e.g., disks.

In the following example, we create a 2D slice of the logarithm of density 500 au wide with the `"top"` direction,
to view the disk from above

In [ ]:
osyris.map(
    {"data": data["hydro"]["density"], "norm": "log"},
    {"data": data["hydro"]["velocity"], "mode": "vec"},
    dx=500 * au,
    origin=center,
    direction="top",
)

We can also use `direction="side"` to view the disk from the side

In [ ]:
osyris.map(
    {"data": data["hydro"]["density"], "norm": "log"},
    {"data": data["hydro"]["velocity"], "mode": "vec"},
    dx=500 * au,
    origin=center,
    direction="side",
)

## Slicing above the origin

We want to plot a slice of density but through a point which is 20 AU above the centre of the domain.
We simply supply a new `origin` to `map`:

In [ ]:
osyris.map(
    {"data": data["hydro"]["density"], "norm": "log"},
    {"data": data["hydro"]["velocity"], "mode": "vec"},
    dx=500 * au,
    origin=center + osyris.Vector(0, 0, 20, unit="au"),
    direction="z",
)

## Plot only a subset of cells belonging to a disk

In this example, we select cells according to their density and plot only those.
This is done by creating a new field and using Numpy's `masked_where` function.
To combine more than one selection criteria, we use the OR operator `|`.

This is useful for plotting disks around protostars, for example.
Here we select the cells with a density in the range
$5 \times 10^{-14}~\text{g cm}^{-3} < \rho < 5 \times 10^{-12}~\text{g cm}^{-3}$.

In [ ]:
condition = (data["hydro"]["density"] < (5.0e-14 * osyris.units("g/cm**3"))) | (
    data["hydro"]["density"] > (5.0e-12 * osyris.units("g/cm**3"))
)
data["hydro"]["disk_density"] = osyris.Array(
    np.ma.masked_where(condition.values, data["hydro"]["density"].values),
    unit=data["hydro"]["density"].unit,
)
osyris.map(data["hydro"]["disk_density"], dx=500 * au, norm="log", origin=center)

Computing the disk mass can be achieved via

In [ ]:
np.sum(data["hydro"]["disk_density"] * (data["amr"]["dx"] ** 3)).to("M_sun")

## Sink particles

Some star formation simulations contain sink particles,
and these can be represented as a scatter layer on a plane plot. 

In [ ]:
osyris.map(
    {"data": data["hydro"]["density"], "norm": "log"},  # layer 1
    {"data": data["sink"]["position"], "mode": "scatter", "c": "white"},  # layer 2
    dx=2000 * au,
    origin=center,
    direction="z",
)

## Subplots / tiled plots

Osyris has no built-in support for subplots (also known as tiled plots).
Instead, we leverage Matplotlib's ability to create such layouts.
Osyris plots are then inserted into the Matplotlib axes, using the `ax` argument.

In the example below, we create four panels and insert various maps.

In [ ]:
# Create figure
fig, ax = plt.subplots(2, 2, figsize=(12, 9))

# Define region to plot
dx = 2000.0 * au

osyris.map(
    {"data": data["hydro"]["density"], "norm": "log"},
    {"data": data["hydro"]["velocity"], "mode": "vec"},
    dx=dx,
    origin=center,
    direction="z",
    ax=ax[0, 0],
)

osyris.map(
    data["hydro"]["velocity"],
    mode="lic",
    color=data["hydro"]["density"],
    norm="log",
    dx=0.25 * dx,
    origin=center,
    direction="z",
    ax=ax[0, 1],
)

osyris.map(
    {
        "data": data["hydro"]["temperature"],
        "norm": "log",
        "mode": "contourf",
        "levels": np.logspace(0.9, 2, 11),
        "cmap": "hot",
    },
    {
        "data": data["amr"]["level"],
        "mode": "contour",
        "colors": "w",
        "levels": [6, 7, 8],
        "fmt": "%i",
    },
    dx=dx,
    origin=center,
    direction="z",
    ax=ax[1, 0],
)

osyris.map(
    {"data": data["hydro"]["density"], "norm": "log", "cmap": "magma"},
    {"data": data["hydro"]["velocity"], "mode": "stream", "color": "w"},
    dx=0.25 * dx,
    origin=center,
    direction="side",
    ax=ax[1, 1],
)